In [1]:
from db import DB

In [2]:
import requests as req
import re

In [3]:
URL_PAT = 'http://auto.smzdm.com/zhengche/p{}/'

In [27]:
reBlock = re.compile(r'<li class="feed-row-wide">([\s\S]*?)<\/li>')
reWorth = re.compile(r'<span class="unvoted-wrap">.*?(\d+)<\/span>')
reTitle = re.compile(r'<a href="http:\/\/www.smzdm.com\/p\/(\d+)\/".*?>([\s\S]*?)<\/a>')
reFavs = re.compile(r'<i class="z-icon-star-empty"><\/i>(\d+)</a>')
reMall = re.compile(r'<a href="\/malllink\/(.*?)"')
reCmts = re.compile(r'<a href="http:\/\/www.smzdm.com\/p\/(\d+)\/\/#comments".*?<\/i>(\d+)<\/a>')

In [28]:
page = 1

In [29]:
url = URL_PAT.format(page)

In [30]:
res = req.get(url)

In [31]:
blocks = reBlock.findall(res.text)

In [32]:
reWorth.findall(blocks[0])

['2', '3']

In [33]:
reTitle.findall(blocks[0])[0]

('8388789',
 '购车必看 : VOLVO 沃尔沃 S60L 线上专享优惠<span class="z-highlight">免购置税、万元置换礼包、首年车险补贴、1460元礼包</span>')

In [34]:
reFavs.findall(blocks[0])

['2']

In [35]:
reMall.findall(blocks[0])

['易车']

In [36]:
reCmts.findall(blocks[0])

[('8388789', '8')]

In [38]:
for page in range(1, 34):
    
    url = URL_PAT.format(page)
    html = req.get(url).text
    
    blocks = reBlock.findall(html)
    for item in blocks:
        record = {}
        votes = reWorth.findall(item)
        title = reTitle.findall(item)[0]
        favs = reFavs.findall(item)
        mall = reMall.findall(item)
        cmts = reCmts.findall(item)
        pid = cmts[0][0]
        comments = int(cmts[0][1])
 
        
        record['up'] = int(votes[0])
        record['down'] = int(votes[1])
        record['pid'] = title[0]
        record['title'] = title[1]
        record['favs'] = int(favs[0])
        record['mall'] = mall[0]
        record['comments'] = comments
       
        DB.insert_one(record)
        
#         DB.update_one({"pid": pid}, {"$set": {"comments": comments}})
        

In [61]:
autos = DB.find({})

In [62]:
for auto in autos:
    title = auto.get("title")
    if ":" in title:
        tag, title = title.split(" : ")
    else:
        tag = "NULL"
        
    
    tt = re.sub(r"<.*?>", '|', title)
    clean_title, highlight, _ = tt.split("|")
    
    DB.update_one({"_id": auto.get("_id")}, {"$set":{"tag": tag, "neat_title": clean_title, "hl": highlight}})
    

In [63]:
import pandas as PD

In [72]:
autos = PD.DataFrame(list(DB.find({}, {"_id": 0, "title": 0})))

In [75]:
autos['NPS'] = (autos['up'] - autos['down'])/(autos['up'] + autos['down'])*100

In [77]:
autos.head()

,comments,down,favs,hl,mall,neat_title,pid,tag,up,NPS
0,9,3,2,免购置税、万元置换礼包、首年车险补贴、1460元礼包,易车,VOLVO 沃尔沃 S60L 线上专享优惠,8388789,购车必看,2,-20.000000
1,8,6,0,千元音乐礼包、易车1460元礼包,易车,CHERY 奇瑞 瑞虎7 线上专享补贴,8388780,购车必看,30,66.666667
2,26,18,7,购车直减10000元、赢取iPhone X,车享,别克 君威 2017款 30H 精英型,8388655,NULL,23,12.195122
3,8,10,6,赢5折购车机会,易车,五菱宝骏 1212 冰点购车节,8385500,购车必看,21,35.483871
4,474,338,74,基准版44.8万起（补贴前）,NIO中国官网,NIO 蔚来 ES8 中大型纯电动SUV,8384763,新车上市,367,4.113475


In [79]:
autos.to_csv("AUTO_SMZDM.csv")